In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

<Axes: xlabel='count', ylabel='domain'>

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<Axes: xlabel='count', ylabel='posted_by'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

/tmp/ipykernel_290682/2178239382.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby(["domain", "posted_by"])


domain         \
                                            count unique   
domain               posted_by                             
americanthinker.com  E. Pluribus Unum          10      1   
                     MtnClimber               190      1   
                     Red Badger                 1      1   
                     SeekAndFind               81      1   
                     george76                   5      1   
apnews.com           E. Pluribus Unum           1      1   
                     Red Badger                 1      1   
                     SeekAndFind                2      1   
                     Libloather                 1      1   
                     hardspunned                2      1   
                     nickcarraway               3      1   
babylonbee.com       george76                   3      1   
                     Morgana                    1      1   
breitbart.com        E. Pluribus Unum          13      1   
                     Red Badger                 6      1   
                     george76                   8      1   
                     hardspunned                1      1   
                     Morgana                   10      1   
                     ChicagoConservative27    839      1   
dailycaller.com      MtnClimber                 6      1   
                     Red Badger                39      1   
                     SeekAndFind                1      1   
                     george76                   9      1   
                     Libloather                 3      1   
                     nickcarraway               1      1   
                     ChicagoConservative27      2      1   
dailymail.co.uk      E. Pluribus Unum           1      1   
                     Red Badger                12      1   
                     SeekAndFind                2      1   
                     george76                  10      1   
                     Libloather                76      1   
                     hardspunned                3      1   
                     Morgana                   60      1   
                     ChicagoConservative27      3      1   
foxnews.com          Red Badger                 2      1   
                     SeekAndFind                2      1   
                     george76                   4      1   
                     Libloather                 4      1   
                     hardspunned                3      1   
                     nickcarraway               1      1   
                     ChicagoConservative27      9      1   
frontpagemag.com     E. Pluribus Unum           9      1   
                     MtnClimber                12      1   
                     Red Badger                 3      1   
                     george76                   7      1   
medicalxpress.com    Red Badger                 1      1   
                     ChicagoConservative27      1      1   
msn.com              E. Pluribus Unum           4      1   
                     Red Badger                 1      1   
                     SeekAndFind               17      1   
                     Libloather                 1      1   
                     nickcarraway               2      1   
                     ChicagoConservative27      1      1   
nbcnews.com          Red Badger                 2      1   
                     SeekAndFind                2      1   
                     george76                   1      1   
                     Libloather                 1      1   
                     hardspunned                2      1   
                     nickcarraway               3      1   
                     Morgana                    4      1   
                     ChicagoConservative27      4      1   
newsweek.com         E. Pluribus Unum           3      1   
                     Red Badger                 2      1   
                     SeekAndFind                1      1   
                     Li

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

/tmp/ipykernel_290682/1921096508.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby('title').


,timestamp,replies
title,,
Shots ring out at PA Trump Rally,2024-07-13 23:35:21,539.0
President Biden Addresses the Nation [LIVE 8PM ET],2024-07-25 02:00:42,413.0
Joe Biden dropping out,2024-07-21 19:52:27,388.0
BREAKING NEWS Shots fired at Donald Trump rally in Pennsylvania,2024-07-13 23:35:21,384.0
Biden Endorses Harris,2024-07-21 20:26:35,338.0
Last Night of the Democratic Committee Convention Live Thread.,2024-08-23 06:20:52,304.0
"Biden tests positive for COVID, cancels speaking engagement",2024-07-17 23:55:51,291.0
Trump claims Kamala Harris rally crowd is FAKE as he posts 'AI doctored photo' - but all is not as it seems,2024-08-12 04:00:11,276.0
Trump Gunman Flew Drone Over Rally Site Hours Before Attempted Assassination,2024-07-20 02:15:05,238.0
